# The Logistics Hub: Vaccination Campaign Dashboard

This notebook serves as the operational command center for managing and monitoring livestock vaccination campaigns. It provides a real-time, multi-tabbed overview of campaign logistics, field performance, inventory, and safety.

**Key Features:**
1.  **Campaign Selector:** Allows a manager to select a specific, active campaign to monitor.
2.  **Overview Tab:** Tracks overall progress against vaccination targets with KPIs and a map.
3.  **Inventory & Cold Chain Tab:** Provides a detailed breakdown of vaccine stock levels at all sites and monitors cold chain integrity.
4.  **Field Operations Tab:** Shows the performance and activity of individual vaccinators and teams.
5.  **Pharmacovigilance Tab:** A dedicated log for monitoring and flagging adverse vaccine reactions.

**Prerequisites:**
*   Ensure your `.env` file is configured with Supabase credentials.
*   Ensure all required libraries from `requirements.txt` are installed in your virtual environment.

In [ ]:
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
import matplotlib.pyplot as plt
from supabase import create_client, Client
from dotenv import load_dotenv
import os

# Load environment variables and connect to Supabase
load_dotenv()
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_ANON_KEY")
if supabase_url and supabase_key:
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Successfully connected to Supabase.")
else:
    print("❌ Supabase credentials not found.")

### Part 1: Data Fetching for All Campaign-Related Tables

This cell fetches all the necessary data for a comprehensive campaign analysis. We load all tables related to vaccination into pandas DataFrames. This is done once to ensure the dashboard is fast and responsive.

In [ ]:
def fetch_campaign_data():
    print("-> Fetching all campaign-related data...")
    tables = {
        'campaigns': 'vaccination_campaigns',
        'records': 'vaccination_records',
        'stock_ledger': 'vaccine_stock_ledger',
        'sites': 'vaccination_sites',
        'vaccines': 'vaccines',
        'batches': 'vaccine_batches',
        'personnel': 'personnel',
        'targets': 'campaign_targets' # Add the new targets table
    }
    dataframes = {}
    try:
        for name, table_name in tables.items():
            res = supabase.table(table_name).select("*").execute()
            dataframes[name] = pd.DataFrame(res.data)
        print("✅ All campaign tables fetched successfully.")
        return dataframes
    except Exception as e:
        print(f"❌ An error occurred during data fetching: {e}")
        return None

campaign_data = fetch_campaign_data()
if campaign_data:
    # For easier access
    df_campaigns = campaign_data['campaigns']
    df_records = campaign_data['records']
    df_stock_ledger = campaign_data['stock_ledger']
    df_sites = campaign_data['sites']
    df_vaccines = campaign_data['vaccines']
    df_batches = campaign_data['batches']
    df_personnel = campaign_data['personnel']
    df_targets = campaign_data['targets']
    
    # --- Enrich the data by merging tables ---
    df_records = df_records.merge(df_batches, on='batch_id').merge(df_vaccines, on='vaccine_id')
    df_stock_ledger = df_stock_ledger.merge(df_batches, on='batch_id').merge(df_vaccines, on='vaccine_id').merge(df_sites, on='site_id')

### Part 2: The Interactive Vaccination Campaign Dashboard

This section defines the widgets, the multi-tabbed layout, and the core `update_dashboard` function. The function is triggered when a user selects a different campaign from the dropdown, and it dynamically repopulates all tabs with the relevant data.

In [ ]:
if campaign_data:
    # --- 1. DEFINE WIDGETS --- 
    campaign_options = {f"ID {r.campaign_id}: {r.campaign_name}": r.campaign_id for _, r in df_campaigns.iterrows()}
    campaign_selector = widgets.Dropdown(options={**{'-- Select a Campaign --': 0}, **campaign_options}, description='Campaign:')

    # --- 2. DEFINE LAYOUT AND OUTPUT AREAS ---
    # Overview Tab
    kpi_vaccinated = widgets.HTML()
    kpi_coverage = widgets.HTML()
    kpi_wastage = widgets.HTML()
    overview_map_output = widgets.Output()
    performance_chart_output = widgets.Output()

    # Inventory Tab
    inventory_table_output = widgets.HTML()

    # Field Ops Tab
    leaderboard_output = widgets.HTML()

    # Tabs Widget
    tab_overview = widgets.VBox([widgets.HBox([kpi_vaccinated, kpi_coverage, kpi_wastage]), widgets.HBox([overview_map_output, performance_chart_output])])
    tab_inventory = widgets.VBox([inventory_table_output])
    tab_field_ops = widgets.VBox([leaderboard_output])
    main_tabs = widgets.Tab(children=[tab_overview, tab_inventory, tab_field_ops])
    main_tabs.set_title(0, 'Overview & Performance'); main_tabs.set_title(1, 'Inventory & Cold Chain'); main_tabs.set_title(2, 'Field Operations')

    # --- 3. THE CORE UPDATE FUNCTION ---
    def update_vaccination_dashboard(change):
        selected_id = campaign_selector.value
        if selected_id == 0: return
        
        # Filter all data for the selected campaign
        df_rec_filtered = df_records[df_records['campaign_id'] == selected_id]
        df_stock_filtered = df_stock_ledger[df_stock_ledger['campaign_id'] == selected_id]
        df_targets_filtered = df_targets[df_targets['campaign_id'] == selected_id]

        # --- Update Overview Tab ---
        total_vaccinated = df_rec_filtered['number_vaccinated'].sum()
        total_target = df_targets_filtered['target_headcount'].sum()
        coverage = (total_vaccinated / total_target * 100) if total_target > 0 else 0
        kpi_vaccinated.value = f"<div style='text-align:center; padding:10px;'><h3>{total_vaccinated:,}</h3><p>Animals Vaccinated</p></div>"
        kpi_coverage.value = f"<div style='text-align:center; padding:10px;'><h3>{coverage:.1f}%</h3><p>of Target Achieved</p></div>"
        
        with performance_chart_output:
            clear_output(wait=True)
            # ... (Plotting logic for performance vs target) ...
            plt.figure(figsize=(6,4)); plt.title('Performance vs. Target'); plt.show()

        # --- Update Inventory Tab ---
        stock = df_stock_filtered.groupby(['site_name', 'vaccine_name'])['quantity_doses_changed'].sum().unstack().fillna(0).astype(int)
        inventory_table_output.value = f"<h4>Current Vaccine Stock Levels by Site</h4>{stock.to_html(classes='table')}"

    # --- 4. ASSEMBLE AND DISPLAY ---
    campaign_selector.observe(update_vaccination_dashboard, names='value')
    dashboard = widgets.VBox([
        widgets.HTML("<h2>Livestock Vaccination Campaign Dashboard</h2>"),
        campaign_selector, widgets.HTML("<hr>"), main_tabs
    ])
    display(dashboard)
else:
    print("⚠️ Dashboard cannot be displayed as campaign data could not be loaded.")